In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from transformers import pipeline
import tweepy
from flair.embeddings import TransformerDocumentEmbeddings
from flair.data import Sentence
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_validate
import nltk
from utils import numerical_df
from baselines import xgboost_baseline, majority_class_baseline, random_class_baseline, ibm_baseline, bert_baseline, bm25_baseline, qa_model_baseline
from sklearn.metrics import f1_score as f1, precision_score as ps, recall_score as rs
from sklearn.model_selection import KFold
from rank_bm25 import BM25Okapi

from sentence_transformers import SentenceTransformer, util

/home/mah/anaconda3/lib/python3.8/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811757556/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


### Hyperparameters

In [19]:
df_arg = numerical_df(pd.read_csv('./mkturk/argumentative.csv')).drop(columns = ['Unnamed: 0'])
df_claim = numerical_df(pd.read_csv('./mkturk/claims.csv'))
## Remove empty strings
#df = df[df.tweet != '']
#df = df[df.tweet.notnull()]

In [23]:
df_arg.loc[df_arg.argumentative > 0.49, 'argumentative'] = 1
df_arg.loc[df_arg.argumentative < 0.5, 'argumentative'] = 0

#df_claim.loc[df_claim.argumentative > 0.49, 'argumentative'] = 1
#df_claim.loc[df_claim.argumentative < 0.5, 'argumentative'] = 0

df_claim.loc[df_claim.claim > 0.49, 'claim'] = 1
df_claim.loc[df_claim.claim < 0.5, 'claim'] = 0



len(df_arg), len(df_claim)

(600, 1271)

In [24]:
current_classes = ['argumentative', 'claim']

In [31]:
display(majority_class_baseline(df_arg, ['argumentative']))
display(majority_class_baseline(df_claim[df_claim.argumentative >= 0.5], ['claim']))

,Task,F1,Precision,Recall
0,argumentative,0.6499999999999999,0.6499999999999999,0.6499999999999999


,Task,F1,Precision,Recall
0,claim,0.7313789455122552,0.7313789455122554,0.7313789455122554


In [32]:
display(random_class_baseline(df_arg, ['argumentative']))
display(random_class_baseline(df_claim[df_claim.argumentative >= 0.5], ['claim']))

,Task,F1,Precision,Recall
0,argumentative,0.4916666666666667,0.4916666666666667,0.4916666666666667


,Task,F1,Precision,Recall
0,claim,0.6078381237876919,0.6078381237876919,0.6078381237876919


In [ ]:
xgboost_baseline(df, tasks = current_classes)

In [40]:
#bm25_baseline(df, current_classes)
#display(bm25_baseline(df_arg, ['argumentative']))
display(bm25_baseline(df_claim[df_claim.argumentative >= 0.5], ['claim']))

/home/mah/.local/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Task,F1,Precision,Recall
0,claim,0.502356,0.502356,0.502356


In [42]:
qa_model_baseline(df_claim[df_claim.argumentative >= 0.5].copy(), ['claim'], model_cutoff = 0.5)

Calculating scores
Loading model


loading configuration file /home/mah/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-MiniLM-L6-cos-v1/config.json
Model config BertConfig {
  "_name_or_path": "/home/mah/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-MiniLM-L6-cos-v1/",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /home/mah/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-MiniLM-L6-cos-v1/pytorch_model.bin
All mo

Scoring tweet topic pair "We should pursue policies that promote sustainable foods"
Loading model


loading configuration file /home/mah/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-MiniLM-L6-cos-v1/config.json
Model config BertConfig {
  "_name_or_path": "/home/mah/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-MiniLM-L6-cos-v1/",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /home/mah/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-MiniLM-L6-cos-v1/pytorch_model.bin
All mo

Scoring tweet topic pair "Meat alternatives should be encouraged"
Loading model


...

loading configuration file /home/mah/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-MiniLM-L6-cos-v1/config.json
Model config BertConfig {
  "_name_or_path": "/home/mah/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-MiniLM-L6-cos-v1/",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /home/mah/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-MiniLM-L6-cos-v1/pytorch_model.bin
All mo

Scoring tweet topic pair "Plant based food should be encouraged"
Loading model


loading configuration file /home/mah/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-MiniLM-L6-cos-v1/config.json
Model config BertConfig {
  "_name_or_path": "/home/mah/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-MiniLM-L6-cos-v1/",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /home/mah/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-MiniLM-L6-cos-v1/pytorch_model.bin
All mo

Scoring tweet topic pair "Vegan and vegetarian diets should be encouraged"


,Task,F1,Precision,Recall
0,claim,0.647502,0.647502,0.647502


In [45]:
#display(ibm_baseline(df_arg, ['argumentative']))
display(ibm_baseline(df_claim[df_claim.argumentative >= 0.5], ['claim']))

Gathering results for claim


ClaimDetectionClient: 100%|██████████| 1061/1061 [00:15<00:00, 66.32it/s] 


,Tasks,F1,Precision,Recall
0,claim,0.328302,0.355102,0.305263


In [47]:
#bert_baseline(df, tasks=current_classes, use_topic=True, batch_size=5, epochs = 1)
#display(ibm_baseline(df_arg, ['argumentative']))
display(bert_baseline(df_claim[df_claim.argumentative >= 0.5], tasks = ['claim'], use_topic=True, batch_size=5, epochs = 1))

Loading tokenizer


loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /home/mah/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/bert-base-cased/resolve/main

Splitting out data, for fold 1


KeyError: 'Passing list-likes to .loc or [] with any missing labels is no longer supported, see https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike'

In [ ]:
df